![uc3m](img/uc3m.jpg)

# Introduction: The Transportation Problem

<a href="http://www.est.uc3m.es/nogales" target="_blank">Javier Nogales</a>

## Statement

We have a set of factories I = [1,2] and set of demand markets J = [1,2,3]. Each market has some product demand $d_j$ and each factory has production capacity $M_i$. There are also transportation costs to deliver one unit from factory i to market j. The goal is to satisfy the demand with minimum cost.

  Distance      | M1 | M2 | M3 | offer
  --------------|-----|---|----|--
  F1  | 2.5 | 1.7 | 1.8 | 350
  F2  | 2.5 | 1.7 | 1.8 | 350
  demand  | 325 | 300 | 275 |

### Import of the Pyomo module

In [8]:
from pyomo.opt import SolverFactory
import sys
from pyomo.environ import *  # import environment
import pyomo.environ as pyo

solvername='glpk'
solverpath_folder='C:\\w64'
solverpath_exe='C:\\w64\\glpsol'

sys.path.append(solverpath_folder)
solver=SolverFactory(solvername, executable=solverpath_exe)

### Optimization framework

Input (data) $\rightarrow$ Model (var, obj, const) $\rightarrow$ Output (solution)

### Define problem data: sets and parameters

Sets are created as attributes object of the main model objects 

Parameter objects are created specifying the sets over which they are defined and are initialised with either a python dictionary or a scalar

In [2]:
# The data

mTransport = ConcreteModel('Transportation Problem')
#Define i and j
mTransport.i = Set(initialize=['P1', 'P2'], doc='origins' )
mTransport.j = Set(initialize=['M1', 'M2', 'M3'], doc='destinations')
#Define capacity and demand
mTransport.Cap = Param(mTransport.i, initialize={'P1' : 350, 'P2': 600 }, doc='origin capacity' )
mTransport.Dem = Param(mTransport.j, initialize={'M1': 325, 'M2': 300, 'M3': 275}, doc='destination demand')

TransportationCost = {
    ('P1', 'M1' ): 2.5,
    ('P1', 'M2'): 1.7,
    ('P1', 'M3' ): 1.8,
    ('P2', 'M1' ): 2.5,
    ('P2', 'M2'): 1.8,
    ('P2', 'M3' ): 1.4
    }


mTransport.Dist = Param(mTransport.i, mTransport.j, initialize=TransportationCost, doc='Distance in hundreds of km')
mTransport.f = 90 # Fixed cost per 100 km in euros

#  Parameter c(i,j)  transport cost in thousands of dollars per case
#            c(i,j) = f * Dist(i,j) 

def cost_euro(mTransport, i, j):
  return mTransport.f * mTransport.Dist[i,j] 

mTransport.Cost = Param(mTransport.i, mTransport.j, initialize=cost_euro, doc='Transportation cost per 100 km in eur')

In [3]:
mTransport.pprint()

4 Set Declarations
    Cost_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :    6 : {('P1', 'M1'), ('P1', 'M2'), ('P1', 'M3'), ('P2', 'M1'), ('P2', 'M2'), ('P2', 'M3')}
    Dist_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    i*j :    6 : {('P1', 'M1'), ('P1', 'M2'), ('P1', 'M3'), ('P2', 'M1'), ('P2', 'M2'), ('P2', 'M3')}
    i : origins
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'P1', 'P2'}
    j : destinations
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'M1', 'M2', 'M3'}

4 Param Declarations
    Cap : origin capacity
        Size=2, Index=i, Domain=Any, Default=None, Mutable=False
        Key : Value
         P1 :   350
         P2 :   600
    Cost : Transportation cost pe

### General formulation for the transportation problem

\begin{align*}
\underset{x_{ij}}{\min} & \quad \sum_{i=1}^{n}\sum_{j=1}^{m}c_{ij}x_{ij}\\
\text{s.t.:}&\\
  &\sum_{i=1}^{n} x_{ij} \geq D_{j} \quad \forall j\\
  &\sum_{j=1}^{m} x_{ij} \leq \bar{C}_{i} \quad \forall i\\
  &x_{ij} \geq 0 \quad \forall ij 
\end{align*}

### 1. Define the variables

In [4]:
# define x(i,j)
mTransport.x = Var(mTransport.i, mTransport.j, doc='units transported', within=NonNegativeReals)

### 2. Define the objective function

In [5]:
def objective_rule(mTransport):
  return sum(mTransport.Cost[i,j]*mTransport.x[i,j] for i in mTransport.i for j in mTransport.j)

mTransport.objective = Objective(rule=objective_rule, sense=minimize, doc='transportation cost')


### 3. Define the constraints

In [6]:
def Capacity(mTransport, i):
  return sum(mTransport.x[i,j] for j in mTransport.j) <= mTransport.Cap[i]

mTransport.Capacity = Constraint(mTransport.i, rule=Capacity, doc='capacity at origin')
    
def Demand (mTransport, j):
  return sum(mTransport.x[i,j] for i in mTransport.i) >= mTransport.Dem[j]

mTransport.Demand = Constraint(mTransport.j, rule=Demand, doc='demand at destination' )



### Let's see the solution

In [9]:
Results = solver.solve(mTransport)

# Display solution
mTransport.x.display()


x : units transported
    Size=6, Index=x_index
    Key          : Lower : Value : Upper : Fixed : Stale : Domain
    ('P1', 'M1') :     0 :  50.0 :  None : False : False : NonNegativeReals
    ('P1', 'M2') :     0 : 300.0 :  None : False : False : NonNegativeReals
    ('P1', 'M3') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('P2', 'M1') :     0 : 275.0 :  None : False : False : NonNegativeReals
    ('P2', 'M2') :     0 :   0.0 :  None : False : False : NonNegativeReals
    ('P2', 'M3') :     0 : 275.0 :  None : False : False : NonNegativeReals


Display solution using pandas

In [10]:
import pandas as pd

transp_results = pd.DataFrame()

for i in mTransport.i:
    for j in mTransport.j:
        transp_results.loc[i,j] = mTransport.x[i,j]()
        
        
print(transp_results)

       M1     M2     M3
P1   50.0  300.0    0.0
P2  275.0    0.0  275.0


### Now the minimum cost

In [11]:
mTransport.objective()

153675.0

### Interpretation

In order to attain the minimum transport costs, 50 units should be sent from P1 to M1, 300 from P1 to M2, 275 from P2 to M1, and 275 from P2 to M3

The total transport costs will be 153675 euros

### We are done

![uc3m](img/wedidit.png)